In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, LabelEncoder, OrdinalEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from lazypredict.Supervised import LazyRegressor

In [2]:
df = pd.read_csv(r"C:\Users\DHONI HANIF\OneDrive\Documents\AI_Collection_and_Loss_Reverse_Forecast\data_preparation\EDA\Univariate_bivariate_multivariate\Univariate_Analysis\new_kolektor.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1000 non-null   int64  
 1   time_to_collect           1000 non-null   int64  
 2   avg_bill_methods          1000 non-null   object 
 3   debtor_volume_handled     1000 non-null   int64  
 4   bill_amount_collected     1000 non-null   int64  
 5   total_actual              1000 non-null   int64  
 6   total_cost                1000 non-null   int64  
 7   success_rate              1000 non-null   float64
 8   collector_age             1000 non-null   int64  
 9   collector_marital_status  1000 non-null   object 
 10  collector_gender          1000 non-null   object 
dtypes: float64(1), int64(7), object(3)
memory usage: 86.1+ KB


In [3]:
for i in df.columns:
    if df[i].dtype == "object":
        df[i] = df[i].astype("category")

df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [4]:
df.head()

,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,collector_age,collector_marital_status,collector_gender
0,0,sms or WA,7,14400000,35700000,277706,40.00,45,Menikah,laki-laki
1,11,panggilan,35,79000000,160500000,22382611,49.00,32,Cerai mati,perempuan
2,4,sms or WA,6,7100000,26700000,205389,27.00,62,Cerai hidup,laki-laki
3,29,datang ke tempat,63,782000000,1395000000,482942548,56.00,55,Cerai mati,perempuan
4,22,datang ke tempat,35,346000000,485500000,151035603,71.00,53,Menikah,perempuan


In [5]:
for i in range(len(df)):
    if df.loc[i, "avg_bill_methods"] == "sms or WA":
        df.loc[i, "debtor_volume_handled"] = np.random.randint(1, 11)
        total = np.random.choice([i for i in np.arange(500000*(df.loc[i, "debtor_volume_handled"]), 10000000*df.loc[i, "debtor_volume_handled"], 100000)])
        df.loc[i, "total_actual"] = total
        bill = np.random.choice([i for i in np.arange(400000*(df.loc[i, "debtor_volume_handled"]), total+100000, 100000)])
        df.loc[i, "bill_amount_collected"] = bill
        df.loc[i, "total_cost"] = np.random.randint(10000*df.loc[i, "debtor_volume_handled"], 50000*(df.loc[i, "debtor_volume_handled"]+1))
        
   
        
df.sample(5)        

,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,collector_age,collector_marital_status,collector_gender
231,14,panggilan,36,213000000,638000000,71961662,33.00,29,Menikah,laki-laki
159,17,datang ke tempat,59,1228000000,1625500000,249362989,76.00,55,Belum menikah,perempuan
599,27,datang ke tempat,46,330000000,620000000,208459802,53.00,61,Cerai mati,laki-laki
222,12,panggilan,34,100000000,139000000,69658970,72.00,42,Cerai mati,perempuan
699,3,sms or WA,7,61300000,63700000,208806,20.00,53,Belum menikah,perempuan


In [6]:
for i in range(len(df)):
    if df.loc[i, "avg_bill_methods"] == "surat panggilan":
        debtor = np.random.randint(11, 21)
        df.loc[i, "debtor_volume_handled"] = debtor
        total = np.random.choice([i for i in np.arange(1000000*debtor, 15000000*debtor, 500000)])
        df.loc[i, "total_actual"] = total
        bill = np.random.choice(np.arange(400000*debtor, total+100000, 100000))
        df.loc[i, "bill_amount_collected"] = bill
        df.loc[i, "total_cost"] = np.random.randint(50000*debtor, 100000*(debtor+1))
    
    elif df.loc[i, "avg_bill_methods"] == "panggilan":
        df.loc[i, "debtor_volume_handled"] =  np.random.randint(21, 26)
        total = np.random.choice([i for i in np.arange(1500000*(df.loc[i, "debtor_volume_handled"]), 20000000*df.loc[i, "debtor_volume_handled"], 1000000)])
        df.loc[i, "total_actual"] = total
        bill = np.random.choice([i for i in np.arange(1000000*(df.loc[i, "debtor_volume_handled"]), total+1000000, 1000000)])
        df.loc[i, "bill_amount_collected"] = bill
        df.loc[i, "total_cost"] = np.random.randint(200000*df.loc[i, "debtor_volume_handled"], 2000000*(df.loc[i, "debtor_volume_handled"]+1))
        
    else:
        df.loc[i, "debtor_volume_handled"] = np.random.randint(26, 31)
        total = np.random.choice([i for i in np.arange(2000000*(df.loc[i, "debtor_volume_handled"]), 35000000*df.loc[i, "debtor_volume_handled"], 1500000)])
        df.loc[i, "total_actual"] = total
        bill = np.random.choice([i for i in np.arange(2000000*(df.loc[i, "debtor_volume_handled"]), total+2000000, 2000000)])
        df.loc[i, "bill_amount_collected"] = bill
        df.loc[i, "total_cost"] = np.random.randint(1000000*df.loc[i, "debtor_volume_handled"], 10000000*(df.loc[i, "debtor_volume_handled"]+1))

df.sample(5)


,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,collector_age,collector_marital_status,collector_gender
734,13,panggilan,21,73000000,101500000,27097400,77.00,58,Cerai hidup,laki-laki
148,2,sms or WA,30,100000000,178500000,253424810,67.00,60,Cerai mati,laki-laki
871,19,datang ke tempat,27,170000000,268500000,47815132,21.00,44,Cerai hidup,laki-laki
417,9,surat panggilan,20,62900000,216500000,1059974,53.00,27,Menikah,perempuan
145,26,datang ke tempat,27,60000000,301500000,220569275,91.00,26,Cerai hidup,laki-laki


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   time_to_collect           1000 non-null   int64   
 1   avg_bill_methods          1000 non-null   category
 2   debtor_volume_handled     1000 non-null   int64   
 3   bill_amount_collected     1000 non-null   int64   
 4   total_actual              1000 non-null   int64   
 5   total_cost                1000 non-null   int64   
 6   success_rate              1000 non-null   float64 
 7   collector_age             1000 non-null   int64   
 8   collector_marital_status  1000 non-null   category
 9   collector_gender          1000 non-null   category
dtypes: category(3), float64(1), int64(6)
memory usage: 58.3 KB


In [8]:
bill_methods = LabelEncoder().fit(df["avg_bill_methods"].to_numpy().reshape(-1, 1))
gender = LabelEncoder().fit(df["collector_gender"].to_numpy().reshape(-1, 1))
mart = LabelEncoder().fit(df["collector_marital_status"].to_numpy().reshape(-1, 1))
df2 = pd.DataFrame()

df2["avg_bill_methods"] = bill_methods.transform(df["avg_bill_methods"])
df2["collector_gender"] = gender.transform(df["collector_gender"])
df2["collector_marital_status"] = mart.transform(df["collector_marital_status"])
           
scaler = RobustScaler().fit(df[["bill_amount_collected", "total_actual"]])
df2[["bill_amount_collected", "total_actual"]] = scaler.transform(df[["bill_amount_collected", "total_actual"]])
df2[["time_to_collect", "debtor_volume_handled", "collector_age", "success_rate"]] = df[["time_to_collect", "debtor_volume_handled", "collector_age", "success_rate"]]
y = df["total_cost"]

df2.head(5)

,avg_bill_methods,collector_gender,collector_marital_status,bill_amount_collected,total_actual,time_to_collect,debtor_volume_handled,collector_age,success_rate
0,2,0,3,0.31,0.12,0,26,45,40.00
1,1,1,2,-0.19,-0.08,11,21,32,49.00
2,2,0,1,-0.26,-0.50,4,26,62,27.00
3,0,1,2,0.10,0.05,29,29,55,56.00
4,0,1,3,0.76,0.39,22,26,53,71.00


In [9]:
X = df2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

  5%|▍         | 2/42 [00:00<00:06,  5.75it/s]

100%|██████████| 42/42 [00:07<00:00,  5.40it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000969 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 710
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 9
[LightGBM] [Info] Start training from score 115127820.160000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
AdaBoostRegressor,0.52,0.54,65334855.38,0.11
BaggingRegressor,0.49,0.51,67129105.04,0.21
RandomForestRegressor,0.48,0.51,67519830.55,1.66
GradientBoostingRegressor,0.47,0.50,68185207.86,0.46
KNeighborsRegressor,0.46,0.49,69020268.94,0.03
ExtraTreesRegressor,0.43,0.46,70682475.16,1.01
OrthogonalMatchingPursuit,0.41,0.44,72213612.25,0.02
OrthogonalMatchingPursuitCV,0.41,0.44,72268717.74,0.03
LassoCV,0.40,0.43,72682623.59,0.16
